In [ ]:
#Descarga de índices bursátiles desde Yahoo Finances.
# -- Sólo descomentar una vez, el resto de usos se pueden hacer desde el fichero descargado
"""
import yfinance as yf

tickers = [
    "^GSPC","^IXIC","^DJI","^RUT",
    "^FTSE","^GDAXI","^FCHI","^125904-USD-STRD","^IBEX",
    "^N225","^HSI","000001.SS","^KS11","^BSESN",
    "^GSPTSE","^BVSP","^MXX","^MERV",
    "^AXJO","^NZ50",
    "ES=F","NQ=F","YM=F","ZT=F","^VIX"
]

# Descarga de cierres diarios sin agrupar por ticker
data = yf.download(
    tickers,
    start="2005-01-01",
    end="2025-01-02"
)

cierres = data["Close"]  # DataFrame con cada ticker como columna
cierres.to_csv("./data/cierres_diarios_2005_2025.csv")
"""

In [1]:
#Uso de los datos descargados para entrenamiento
# -- Asegurarse de enrutamiento y nombre de fichero correctos

import pandas as pd
df = pd.read_csv('./data/cierres_diarios_2005_2025n.csv', parse_dates=['Date'], index_col='Date')

In [2]:
# Preparación de los datos

# Eliminación de nulos
df.ffill(inplace=True)
df.bfill(inplace=True)

# Split en crudo (¡antes de escalar!)
n = len(df)
train_raw = df.iloc[:int(n*0.7)]
val_raw   = df.iloc[int(n*0.7):int(n*0.9)]
test_raw  = df.iloc[int(n*0.9):]

# Ajustar scaler SOLO con TRAIN
from sklearn.preprocessing import MinMaxScaler
from joblib import dump
scaler = MinMaxScaler()
scaler.fit(train_raw)  # <- fit solo con train

# Transformar cada split con ese scaler
import pandas as pd
train_df = pd.DataFrame(scaler.transform(train_raw), index=train_raw.index, columns=train_raw.columns).astype("float32")
val_df   = pd.DataFrame(scaler.transform(val_raw),   index=val_raw.index,   columns=val_raw.columns).astype("float32")
test_df  = pd.DataFrame(scaler.transform(test_raw),  index=test_raw.index,  columns=test_raw.columns).astype("float32")

# Guardar scaler para escenarios S/M/L
dump(scaler, "scaler_modelos.joblib")


import tensorflow as tf

def make_dataset(data, window_size, horizon, batch_size=32, shuffle=True):
    ds = tf.keras.preprocessing.timeseries_dataset_from_array(
        data=data.values,
        targets=None,
        sequence_length=window_size + horizon,
        sequence_stride=1,
        batch_size=batch_size,
        shuffle=shuffle
    )
    return ds.map(
        lambda seq: (
            tf.cast(seq[:, :window_size, :], tf.float32),
            tf.cast(seq[:, window_size:, :], tf.float32)
        )
    )


# Recreamos datasets con shuffle=False para val/test:
train_dss = make_dataset(train_df, window_size=20,  horizon=1,  batch_size=32, shuffle=True)
val_dss   = make_dataset(val_df,   window_size=20,  horizon=1,  batch_size=32, shuffle=False)
test_dss  = make_dataset(test_df,  window_size=20,  horizon=1,  batch_size=32, shuffle=False)

train_dsm = make_dataset(train_df, window_size=60,  horizon=5,  batch_size=32, shuffle=True)
val_dsm   = make_dataset(val_df,   window_size=60,  horizon=5,  batch_size=32, shuffle=False)
test_dsm  = make_dataset(test_df,  window_size=60,  horizon=5,  batch_size=32, shuffle=False)

train_dsl = make_dataset(train_df, window_size=120, horizon=20, batch_size=32, shuffle=True)
val_dsl   = make_dataset(val_df,   window_size=120, horizon=20, batch_size=32, shuffle=False)
test_dsl  = make_dataset(test_df,  window_size=120, horizon=20, batch_size=32, shuffle=False)

2025-09-19 08:46:44.243585: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-09-19 08:46:44.243614: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-09-19 08:46:44.243621: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-09-19 08:46:44.243637: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-19 08:46:44.243646: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# =========================================================
# CNN + Huber (sweep delta)
# Selección de δ basada en métrica común: val_log_cosh
# Grid de δ calibrado con cuantiles de |e| (baseline persistencia) en VAL
# =========================================================
import os
import numpy as np
import tensorflow as tf
from typing import List, Tuple
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input, Conv1D, Dropout, Dense, Reshape, LayerNormalization
)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# ---------- Scaler único (MinMax) ----------
try:
    scaler
except NameError:
    from joblib import load
    scaler = load("scaler_modelos.joblib")

# ---------- Config ----------
MODEL_DIR = "./models_cnn_huber_sweep"
os.makedirs(MODEL_DIR, exist_ok=True)

# EPS para métricas within (escala normalizada [0,1])
EPS_LIST = [0.005, 0.01, 0.02, 0.05, 0.1]

# ---------- Utils ----------
def infer_shapes_from_dataset(ds: tf.data.Dataset) -> Tuple[int, int, int]:
    for xb, yb in ds.take(1):
        w = int(xb.shape[1])
        f = int(xb.shape[2])
        h = int(yb.shape[1])
        return w, f, h
    raise ValueError("Dataset vacío")

def _eps_tag(x: float) -> str:
    s = f"{x:.10g}"
    s = s.rstrip('0').rstrip('.') if '.' in s else s
    return s.replace('.', '_')

# ---------- Métricas ----------
def log_cosh_metric(y_true, y_pred):
    e = tf.cast(y_pred, tf.float32) - tf.cast(y_true, tf.float32)
    ae = tf.abs(e)
    return tf.reduce_mean(ae + tf.nn.softplus(-2.0 * ae) - tf.math.log(2.0))
log_cosh_metric.__name__ = "log_cosh"

def make_within_eps_vector_metric(eps_vec: np.ndarray, tag: str):
    eps_tf = tf.constant(eps_vec.astype(np.float32), dtype=tf.float32)  # (F,)
    def within_eps(y_true, y_pred):
        diff = tf.abs(tf.cast(y_pred, tf.float32) - tf.cast(y_true, tf.float32))  # (B,H,F)
        thr  = eps_tf[tf.newaxis, tf.newaxis, :]
        hit  = tf.cast(diff <= thr, tf.float32)
        return tf.reduce_mean(hit)
    within_eps.__name__ = tag
    return within_eps

def build_within_metrics_minmax(scaler, eps_list: List[float], n_features: int):
    if not hasattr(scaler, "data_range_"):
        raise ValueError("Se esperaba MinMaxScaler con data_range_.")
    if len(scaler.data_range_) != n_features:
        raise ValueError("scaler.data_range_ no coincide con n_features.")
    metrics = [log_cosh_metric]
    for e in eps_list:
        eps_vec = np.full((n_features,), float(e), dtype=np.float32)
        tag = f"within_eps_{_eps_tag(e)}"
        metrics.append(make_within_eps_vector_metric(eps_vec, tag))
    return metrics

def _get_metric_value_relaxed(res: dict, base_name: str):
    """
    Devuelve res[base_name] si existe; si no, busca cualquier clave que empiece por base_name + '_'
    (para cubrir sufijos que Keras añade como '_1', '_2', etc.). Si no hay match, devuelve NaN.
    """
    if base_name in res and np.isfinite(res[base_name]):
        return float(res[base_name])
    # Busca con sufijo
    for k, v in res.items():
        if k.startswith(base_name + "_"):
            try:
                v = float(v)
                if np.isfinite(v):
                    return v
            except Exception:
                pass
    return float("nan")

def compute_autc_from_results(res: dict, eps_list) -> float:
    eps = np.array(sorted(eps_list), dtype=np.float64)
    acc = []
    missing = []
    for e in eps:
        base = f"within_eps_{_eps_tag(e)}"
        val = _get_metric_value_relaxed(res, base)
        acc.append(val)
        if not np.isfinite(val):
            missing.append(base)
    acc = np.array(acc, dtype=np.float64)

    # Debug útil si algo falla
    if missing:
        print("[AUTC] Aviso: no se encontraron métricas:", missing)
        print("[AUTC] Claves disponibles:", sorted([k for k in res.keys() if "within_eps_" in k]))

    mask = np.isfinite(acc)
    if mask.sum() < 2:
        return float("nan")

    return float(np.trapz(acc[mask], eps[mask]) / (eps[mask][-1] - eps[mask][0]))


# ---------- Modelo CNN (sin Flatten; Reshape explícito) ----------
def build_cnn_point_model(window: int, n_features: int, horizon: int,
                          f1: int = 128, k1: int = 5, d1: float = 0.2,
                          f2: int = 64,  k2: int = 3, d2: float = 0.2) -> Model:
    """
    Conv1D causal -> LN -> Dropout -> Conv1D causal -> LN -> Dropout
    -> Reshape((window*f2,)) -> Dense(horizon*n_features) -> Reshape((horizon, n_features))
    """
    inp = Input(shape=(window, n_features))
    x = Conv1D(f1, k1, padding="causal", activation="relu")(inp)
    x = LayerNormalization()(x)
    x = Dropout(d1)(x)

    x = Conv1D(f2, k2, padding="causal", activation="relu")(x)
    x = LayerNormalization()(x)
    x = Dropout(d2)(x)

    x = Reshape((window * f2,))(x)
    x = Dense(horizon * n_features)(x)
    out = Reshape((horizon, n_features))(x)
    return Model(inp, out, name=f"CNN_POINT_H{horizon}_F{n_features}")

# ---------- Pérdida Huber ----------
def make_huber_loss(delta: float):
    base = tf.keras.losses.Huber(delta=float(delta))
    def huber_loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32); y_pred = tf.cast(y_pred, tf.float32)
        return base(y_true, y_pred)
    huber_loss.__name__ = f"huber_delta_{_eps_tag(delta)}"
    return huber_loss

# ---------- Calibración del grid de δ (cuantiles de |e| con baseline persistencia) ----------
def estimate_error_quantiles_persistence(val_ds: tf.data.Dataset, max_batches: int = 256):
    """Cuantiles de |e| en VAL usando baseline de persistencia (y_hat = último paso repetido)."""
    errs = []
    taken = 0
    for xb, yb in val_ds:
        yhat = tf.repeat(xb[:, -1:, :], repeats=tf.shape(yb)[1], axis=1)  # (B,H,F)
        e = tf.abs(tf.cast(yb, tf.float32) - tf.cast(yhat, tf.float32)).numpy().ravel()
        errs.append(e)
        taken += 1
        if taken >= max_batches:
            break
    if not errs:
        return [0.01, 0.02, 0.05, 0.1]
    e = np.concatenate(errs)
    qs = np.quantile(e, [0.5, 0.75, 0.9, 0.95])  # p50, p75, p90, p95
    return list(qs)

def build_delta_grid(val_ds: tf.data.Dataset):
    base = [0.01, 0.02, 0.05, 0.1]
    qs = estimate_error_quantiles_persistence(val_ds, max_batches=256)
    cand = sorted(set(base + qs))
    cand = [float(np.clip(c, 1e-4, 0.5)) for c in cand]
    cand = sorted(set(cand))
    print("\nGrid δ (calibrado con cuantiles |e| en VAL):", cand)
    return cand

# ---------- Entrenamiento para un δ (callbacks en val_log_cosh) ----------
def train_for_delta(train_ds: tf.data.Dataset,
                    val_ds: tf.data.Dataset,
                    delta: float,
                    scenario_tag: str,
                    scaler) -> tuple[Model, float, str]:
    w, f, h = infer_shapes_from_dataset(train_ds)
    model = build_cnn_point_model(w, f, h)
    metrics = build_within_metrics_minmax(scaler, EPS_LIST, n_features=f)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4, clipnorm=1.0),
        loss=make_huber_loss(delta),
        metrics=metrics
    )

    ckpt_path = os.path.join(
        MODEL_DIR, f"cnn_huber_w{w}_h{h}_delta{_eps_tag(delta)}{scenario_tag}.keras"
    )

    callbacks = [
        tf.keras.callbacks.TerminateOnNaN(),
        EarlyStopping(monitor="val_log_cosh", mode="min", patience=15, restore_best_weights=True),
        ModelCheckpoint(ckpt_path, monitor="val_log_cosh", mode="min", save_best_only=True),
    ]

    hist = model.fit(train_ds, validation_data=val_ds, epochs=160, callbacks=callbacks, verbose=1)
    vlc = np.array(hist.history.get("val_log_cosh", []), dtype=np.float32)
    vlc = vlc[np.isfinite(vlc)]
    best_val_logcosh = float(np.min(vlc)) if vlc.size > 0 else np.inf

    vll = np.array(hist.history.get("val_loss", []), dtype=np.float32)
    vll = vll[np.isfinite(vll)]
    best_val_huber = float(np.min(vll)) if vll.size > 0 else np.inf
    
    return model, best_val_logcosh, best_val_huber, ckpt_path

# ---------- Barrido de deltas (selección por val_log_cosh) ----------
def sweep_deltas(train_ds, val_ds, scenario_tag: str, scaler) -> tuple[Model, float, str]:
    deltas = build_delta_grid(val_ds)
    print(f"\n=== Barrido Huber delta {scenario_tag} (CNN) ===")
    results = []  # (delta, best_val_logcosh, best_val_huber, ckpt_path)

    best_score = np.inf; best_ckpt=None; best_delta=None; best_model=None
    
    for d in deltas:
        print(f"\n--- Entrenando δ={d} ---")
        model, val_logcosh, val_huber, ckpt_path = train_for_delta(train_ds, val_ds, d, scenario_tag, scaler)
        print(f"  -> min val_log_cosh={val_logcosh:.12f} | min val_huber={val_huber:.12f}")
        results.append((d, val_logcosh, val_huber, ckpt_path))
        if val_logcosh < best_score:  # selección por métrica común
            best_score, best_ckpt, best_delta, best_model = val_logcosh, ckpt_path, d, model

    # Carga el mejor checkpoint (Flask-ready)
    if best_ckpt:
        best_model = tf.keras.models.load_model(best_ckpt, compile=False)

    # Tabla ordenada para inspección
    results_sorted = sorted(results, key=lambda x: x[1])
    print("\nTabla val_log_cosh(min) por δ (orden asc):")
    for d, v_log, v_hub, _ in results_sorted:
        print(f"  δ={d:>8g}: val_log_cosh={v_log:.12f} | val_huber={v_hub:.12f}")
    print(f"\n>>> Mejor δ por val_log_cosh: {best_delta} (val_log_cosh={best_score:.12f})")
    return best_model, best_delta, best_ckpt

# ---------- Evaluación en TEST ----------
def evaluate_on_test(model: tf.keras.Model, ds: tf.data.Dataset, best_delta: float, scaler):
    if (model is None) or (best_delta is None) or (not np.isfinite(best_delta)):
        print("  [AVISO] No se pudo entrenar un modelo válido.")
        return
    _, f, _ = infer_shapes_from_dataset(ds)
    metrics = build_within_metrics_minmax(scaler, EPS_LIST, n_features=f)
    model.compile(optimizer="adam", loss=make_huber_loss(best_delta), metrics=metrics)

    res = model.evaluate(ds, return_dict=True, verbose=0)
    print("  loss (Huber):              {:.6f}".format(res.get("loss", float("nan"))))
    print("  log_cosh:                  {:.6f}".format(res.get("log_cosh", float("nan"))))
    for e in EPS_LIST:
        key = f"within_eps_{_eps_tag(e)}"
        print(f"  {key:26s}: {res.get(key, float('nan')):.6f}")
    autc = compute_autc_from_results(res, EPS_LIST)
    eps_min, eps_max = float(min(EPS_LIST)), float(max(EPS_LIST))
    print(f"  AUTC[{eps_min:.3f}–{eps_max:.3f}]:         {autc:.6f}")

# ================== EJECUCIÓN: TRES ESCENARIOS ==================
# Usa tus datasets ya creados (sin shuffle en val/test):
#   train_dss/val_dss/test_dss, train_dsm/val_dsm/test_dsm, train_dsl/val_dsl/test_dsl

# Escenario S (20→1)
model_s, delta_s, path_s = sweep_deltas(train_dss, val_dss, scenario_tag="_S", scaler=scaler)
print("\nResultados TEST - Escenario S (CNN)")
evaluate_on_test(model_s, test_dss, best_delta=delta_s, scaler=scaler)

# Escenario M (60→5)
model_m, delta_m, path_m = sweep_deltas(train_dsm, val_dsm, scenario_tag="_M", scaler=scaler)
print("\nResultados TEST - Escenario M (CNN)")
evaluate_on_test(model_m, test_dsm, best_delta=delta_m, scaler=scaler)

# Escenario L (120→20)
model_l, delta_l, path_l = sweep_deltas(train_dsl, val_dsl, scenario_tag="_L", scaler=scaler)
print("\nResultados TEST - Escenario L (CNN)")
evaluate_on_test(model_l, test_dsl, best_delta=delta_l, scaler=scaler)

print("\n=== Mejor δ por escenario (CNN, métrica común: val_log_cosh) ===")
print(f"  S: {delta_s}  -> {path_s}")
print(f"  M: {delta_m}  -> {path_m}")
print(f"  L: {delta_l}  -> {path_l}")



Grid δ (calibrado con cuantiles |e| en VAL): [0.00853496789932251, 0.01, 0.017357412725687027, 0.02, 0.03049677610397339, 0.04116208553314208, 0.05, 0.1]

=== Barrido Huber delta _S (CNN) ===

--- Entrenando δ=0.00853496789932251 ---
Epoch 1/160


2025-09-19 08:46:46.344307: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-09-19 08:46:46.830019: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


114/114 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - log_cosh: 0.3137 - loss: 0.0060 - within_eps_0_005: 0.0051 - within_eps_0_01: 0.0095 - within_eps_0_02: 0.0179 - within_eps_0_05: 0.0453 - within_eps_0_1: 0.0912 - val_log_cosh: 0.0987 - val_loss: 0.0029 - val_within_eps_0_005: 0.0081 - val_within_eps_0_01: 0.0173 - val_within_eps_0_02: 0.0356 - val_within_eps_0_05: 0.0890 - val_within_eps_0_1: 0.1940
Epoch 2/160
114/114 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - log_cosh: 0.1753 - loss: 0.0043 - within_eps_0_005: 0.0066 - within_eps_0_01: 0.0128 - within_eps_0_02: 0.0246 - within_eps_0_05: 0.0634 - within_eps_0_1: 0.1251 - val_log_cosh: 0.0879 - val_loss: 0.0026 - val_within_eps_0_005: 0.0135 - val_within_eps_0_01: 0.0242 - val_within_eps_0_02: 0.0500 - val_within_eps_0_05: 0.1303 - val_within_eps_0_1: 0.2527
Epoch 3/160
114/114 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - log_cosh: 0.1086 - loss: 0.0033 - within_eps_0_005: 0.0075 - within_eps_0_01: 0.0156 - within_eps_0_02: 0.0322 - within_eps_0_05: 0.0810

2025-09-19 08:56:56.890248: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


112/112 ━━━━━━━━━━━━━━━━━━━━ 8s 48ms/step - log_cosh: 0.3265 - loss: 0.0072 - within_eps_0_005: 0.0045 - within_eps_0_01: 0.0093 - within_eps_0_02: 0.0186 - within_eps_0_05: 0.0470 - within_eps_0_1: 0.0926 - val_log_cosh: 0.0858 - val_loss: 0.0031 - val_within_eps_0_005: 0.0120 - val_within_eps_0_01: 0.0240 - val_within_eps_0_02: 0.0483 - val_within_eps_0_05: 0.1227 - val_within_eps_0_1: 0.2407
Epoch 2/160
112/112 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - log_cosh: 0.1468 - loss: 0.0045 - within_eps_0_005: 0.0073 - within_eps_0_01: 0.0144 - within_eps_0_02: 0.0281 - within_eps_0_05: 0.0704 - within_eps_0_1: 0.1390 - val_log_cosh: 0.0728 - val_loss: 0.0027 - val_within_eps_0_005: 0.0126 - val_within_eps_0_01: 0.0255 - val_within_eps_0_02: 0.0502 - val_within_eps_0_05: 0.1324 - val_within_eps_0_1: 0.2678
Epoch 3/160
112/112 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - log_cosh: 0.0593 - loss: 0.0027 - within_eps_0_005: 0.0121 - within_eps_0_01: 0.0238 - within_eps_0_02: 0.0474 - within_eps_0_05: 0.1170

2025-09-19 09:12:18.496116: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


110/110 ━━━━━━━━━━━━━━━━━━━━ 10s 64ms/step - log_cosh: 0.3178 - loss: 0.0071 - within_eps_0_005: 0.0047 - within_eps_0_01: 0.0094 - within_eps_0_02: 0.0188 - within_eps_0_05: 0.0470 - within_eps_0_1: 0.0936 - val_log_cosh: 0.0952 - val_loss: 0.0033 - val_within_eps_0_005: 0.0103 - val_within_eps_0_01: 0.0204 - val_within_eps_0_02: 0.0408 - val_within_eps_0_05: 0.1017 - val_within_eps_0_1: 0.1994
Epoch 2/160
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - log_cosh: 0.1675 - loss: 0.0049 - within_eps_0_005: 0.0064 - within_eps_0_01: 0.0129 - within_eps_0_02: 0.0256 - within_eps_0_05: 0.0641 - within_eps_0_1: 0.1281 - val_log_cosh: 0.0810 - val_loss: 0.0030 - val_within_eps_0_005: 0.0117 - val_within_eps_0_01: 0.0236 - val_within_eps_0_02: 0.0465 - val_within_eps_0_05: 0.1138 - val_within_eps_0_1: 0.2263
Epoch 3/160
110/110 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - log_cosh: 0.0997 - loss: 0.0037 - within_eps_0_005: 0.0086 - within_eps_0_01: 0.0171 - within_eps_0_02: 0.0340 - within_eps_0_05: 0.085